<a href="https://colab.research.google.com/github/giabao993255/luanvanhk2/blob/main/luanvan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tinh chỉnh LayoutLM bằng tập dữ liệu SROIE
Sổ tay này là nỗ lực của tôi trong việc tinh chỉnh mô hình LayoutLM bằng cách sử dụng tập dữ liệu SROIE để trích xuất thông tin. Mô hình được trình bày trong bài báo "[LayoutLM: Đào tạo trước về Văn bản và Bố cục để Hiểu Hình ảnh Tài liệu] (https://arxiv.org/abs/1912.13318)" của Yiheng Xu, Minghao Li, Lei Cui, Shaohan Huang, Furu Wei và Ming Zhou và máy tính xách tay đã được truyền cảm hứng rất nhiều từ máy tính xách tay "ruifcruz" của người dùng Github có tiêu đề "Tinh chỉnh SROIE trên LayoutLM".
- LayoutLM Github repo [tại đây] (https://github.com/microsoft/unilm/tree/master/layoutlm).
- Bộ dữ liệu được lấy từ cuộc thi ICDAR 2019 SROIE [tại đây] (https://rrc.cvc.uab.es/?ch=13).
- "Tinh chỉnh SROIE trên LayoutLM" của ruifcruz [tại đây] (https://github.com/ruifcruz/sroie-on-layoutlm).

## Ghi chú:
- Máy tính xách tay đang sử dụng tập lệnh tiền xử lý được bao gồm trong repo của mô hình [tại đây] (https://github.com/microsoft/unilm/tree/master/layoutlm/examples/seq_labeling) ban đầu được sử dụng để tinh chỉnh tập dữ liệu FUNSD. Chúng ta có thể đào tạo trên tập dữ liệu SROIE với cùng một tập lệnh vì nó dành cho việc ghi nhãn trình tự.
- Các tác giả ban đầu của mô hình nhận được điểm F1 ** 0,946 ** (sử dụng mô hình cơ sở) và điểm tối đa tôi nhận được là điểm F1 ** 0,957 ** (10 kỷ nguyên). Đây là một sự cải thiện so với điểm của ruifcruz, bởi vì tôi cũng bao gồm nhãn "địa chỉ công ty". Đó cũng là một sự cải thiện về điểm số của các tác giả và lý do cho điều đó, như được giải thích bởi ruifcruz, có thể là:
  - Các siêu tham số khác nhau,
  - OCR chất lượng - các tác giả không sử dụng bản quét SROIE OCR mà tạo ra bản quét của riêng họ,
  - tiền xử lý tốt hơn
  

# 1. Xử lý trước tập dữ liệu
Trước khi tinh chỉnh mô hình, chúng tôi phải xử lý trước tập dữ liệu SROIE có thể tải xuống từ [tại đây] (https://drive.google.com/drive/folders/1ShItNWXyiY1tFDM5W02bceHuJjyeeJl2). Tập dữ liệu chứa nhiều thư mục con, vì cuộc thi được chia thành ba nhiệm vụ ** Bản địa hóa văn bản, Nhận dạng ký tự quang học (OCR) ** và ** Trích xuất thông tin (IE) ** và một số thư mục dành cho nhiệm vụ cụ thể của chúng. Vì mục đích của chúng tôi, chúng tôi chỉ quan tâm đến nhiệm vụ cuối cùng, vì vậy các thư mục dành cho nó là:
- ** 0325updated.task1train (626p) ** - chứa hình ảnh biên lai (.jpg) và các hộp giới hạn OCR'd tương ứng và văn bản (.txt)
- ** 0325updated.task2train (626p) ** - chứa văn bản được gắn nhãn (.txt) ở định dạng JSON.

Nhưng đối với sổ ghi chép này, tôi thực sự sẽ sử dụng một phiên bản có tổ chức của bộ datset có sẵn công khai mà tôi đã tạo. Tập dữ liệu được tổ chức theo cách dễ hiểu, không có bất kỳ tệp trùng lặp nào và tôi cũng đã thêm một tập hợp con thử nghiệm không có trong liên kết trước đó. Tập dữ liệu có sẵn [tại đây] (https://www.kaggle.com/urbikn/sroie-datasetv2).

In [ ]:
import os
import glob
import json 
import random
from pathlib import Path
from difflib import SequenceMatcher


import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from matplotlib import pyplot, patches

## Chuẩn bị tập dữ liệu
Vị trí của tập dữ liệu SROIE và tên của tệp ví dụ được sử dụng cho mục đích trình diễn

In [ ]:
sroie_folder_path = Path('/kaggle/input/sroie-datasetv2/SROIE2019')
example_file = Path('X51005365187.txt')

### Đọc các từ và hộp giới hạn
Vì vậy, bước đầu tiên là đọc dữ liệu OCR, trong đó mỗi dòng trong tệp bao gồm một nhóm từ và một hộp giới hạn xác định chúng. Tất cả những gì chúng ta phải làm là đọc tệp, loại bỏ các điểm không cần thiết trong hộp giới hạn (vì mô hình chỉ yêu cầu các điểm trên cùng bên trái và dưới cùng bên phải) và lưu nó trong Pandas Dataframe.

In [ ]:
def read_bbox_and_words(path: Path):
  bbox_and_words_list = []

  with open(path, 'r', errors='ignore') as f:
    for line in f.read().splitlines():
      if len(line) == 0:
        continue
        
      split_lines = line.split(",")

      bbox = np.array(split_lines[0:8], dtype=np.int32)
      text = ",".join(split_lines[8:])

      # From the splited line we save (filename, [bounding box points], text line).
      # The filename will be useful in the future
      bbox_and_words_list.append([path.stem, *bbox, text])
    
  dataframe = pd.DataFrame(bbox_and_words_list, columns=['filename', 'x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'line'], dtype=np.int16)
  dataframe = dataframe.drop(columns=['x1', 'y1', 'x3', 'y3'])

  return dataframe


# Example usage
bbox_file_path = sroie_folder_path / "test/box" / example_file
print("== File content ==")
!head -n 5 "{bbox_file_path}"

bbox = read_bbox_and_words(path=bbox_file_path)
print("\n== Dataframe ==")
bbox.head(5)

### Đọc tệp thực thể
Bây giờ chúng ta cần đọc tệp thực thể để biết những gì cần gắn nhãn trong văn bản của chúng ta.

In [ ]:
def read_entities(path: Path):
  with open(path, 'r') as f:
    data = json.load(f)

  dataframe = pd.DataFrame([data])
  return dataframe


# Example usage
entities_file_path = sroie_folder_path /  "test/entities" / example_file
print("== File content ==")
!head "{entities_file_path}"

entities = read_entities(path=entities_file_path)
print("\n\n== Dataframe ==")
entities

### Gán nhãn cho các từ bằng cách sử dụng dữ liệu thực thể
Chúng ta có các từ / dòng và thực thể, bây giờ chúng ta chỉ cần kết hợp chúng lại với nhau bằng cách gắn nhãn các dòng của chúng ta bằng cách sử dụng các giá trị thực thể. Chúng tôi sẽ thực hiện điều đó bằng cách kết hợp các chuỗi con với các giá trị thực thể với các dòng và nếu chúng không khớp với một kiểm tra độ tương đồng bằng cách sử dụng pythons _difflib.SequenceMatcher_ và chỉ định bất kỳ thứ gì cao hơn so với dự đoán 0,8 (80%).

** Nhãn "O" ** sẽ xác định tất cả các từ không được gắn nhãn của chúng ta trong bước gán, vì chúng ta bắt buộc phải gắn nhãn cho mọi thứ.

In [ ]:
# Assign a label to the line by checking the similarity
# of the line and all the entities
def assign_line_label(line: str, entities: pd.DataFrame):
    line_set = line.replace(",", "").strip().split()
    for i, column in enumerate(entities):
        entity_values = entities.iloc[0, i].replace(",", "").strip()
        entity_set = entity_values.split()
        
        
        matches_count = 0
        for l in line_set:
            if any(SequenceMatcher(a=l, b=b).ratio() > 0.8 for b in entity_set):
                matches_count += 1
            
            if (column.upper() == 'ADDRESS' and (matches_count / len(line_set)) >= 0.5) or \
               (column.upper() != 'ADDRESS' and (matches_count == len(line_set))) or \
               matches_count == len(entity_set):
                return column.upper()

    return "O"


line = bbox.loc[1,"line"]
label = assign_line_label(line, entities)
print("Line:", line)
print("Assigned label:", label)

Với một chức năng có thể xử lý việc ghi nhãn các dòng của chúng tôi, chúng tôi sẽ tạo một chức năng khác để gắn nhãn tất cả các dòng của chúng tôi trong một DataFrame (vì vậy một biên nhận).

Đơn giản như điều này có thể xảy ra, vấn đề nảy sinh khi chúng ta nhận được các dòng mà tất cả sẽ vượt qua cùng một đối sánh, như ** TOTAL ** chẳng hạn; biên lai có thể chỉ có một mặt hàng trên đó và giá của nó có thể bằng với tổng cuối cùng, do đó, các nhãn trùng lặp. Hoặc có thể một phần của địa chỉ cũng có ở cuối biên nhận.

Để bỏ qua các ví dụ như vậy, tôi đã viết các quy tắc đơn giản được mã hóa cứng để chỉ định * tổng số * và * ngày * cho chỉ các hộp giới hạn lớn nhất mà nó có thể tìm thấy (dựa trên khu vực của nó) và không cho phép chỉ định địa chỉ sau ngày hoặc tổng số.

In [ ]:
def assign_labels(words: pd.DataFrame, entities: pd.DataFrame):
    max_area = {"TOTAL": (0, -1), "DATE": (0, -1)}  # Value, index
    already_labeled = {"TOTAL": False,
                       "DATE": False,
                       "ADDRESS": False,
                       "COMPANY": False,
                       "O": False
    }

    # Go through every line in $words and assign it a label
    labels = []
    for i, line in enumerate(words['line']):
        label = assign_line_label(line, entities)

        already_labeled[label] = True
        if (label == "ADDRESS" and already_labeled["TOTAL"]) or \
           (label == "COMPANY" and (already_labeled["DATE"] or already_labeled["TOTAL"])):
            label = "O"

        # Assign to the largest bounding box
        if label in ["TOTAL", "DATE"]:
            x0_loc = words.columns.get_loc("x0")
            bbox = words.iloc[i, x0_loc:x0_loc+4].to_list()
            area = (bbox[2] - bbox[0]) + (bbox[3] - bbox[1])

            if max_area[label][0] < area:
                max_area[label] = (area, i)

            label = "O"

        labels.append(label)

    labels[max_area["DATE"][1]] = "DATE"
    labels[max_area["TOTAL"][1]] = "TOTAL"

    words["label"] = labels
    return words


# Example usage
bbox_labeled = assign_labels(bbox, entities)
bbox_labeled.head(15)

### Chia từ
Đối với phần cuối cùng, chúng tôi chia các dòng thành các mã thông báo riêng biệt với các hộp giới hạn của riêng chúng.

Tách các hộp giới hạn dựa trên độ dài của từ có lẽ không phải là cách tốt nhất, nhưng nó đủ tốt.

In [ ]:
def split_line(line: pd.Series):
  line_copy = line.copy()

  line_str = line_copy.loc["line"]
  words = line_str.split(" ")

  # Filter unwanted tokens
  words = [word for word in words if len(word) >= 1]

  x0, y0, x2, y2 = line_copy.loc[['x0', 'y0', 'x2', 'y2']]
  bbox_width = x2 - x0
  

  new_lines = []
  for index, word in enumerate(words):
    x2 = x0 + int(bbox_width * len(word)/len(line_str))
    line_copy.at['x0', 'x2', 'line'] = [x0, x2, word]
    new_lines.append(line_copy.to_list())
    x0 = x2 + 5 

  return new_lines


# Example usage
new_lines = split_line(bbox_labeled.loc[1])
print("Original row:")
display(bbox_labeled.loc[1:1,:])

print("Splitted row:")
pd.DataFrame(new_lines, columns=bbox_labeled.columns)

### Để tất cả chúng cùng nhau
Chúng tôi đã xác định tất cả các chức năng của mình, bây giờ chúng tôi chỉ cần sử dụng chúng trên mọi tệp và chuyển đổi tập dữ liệu thành một định dạng mà tập lệnh / mô hình có thể phân tích cú pháp.

In [ ]:
from time import perf_counter
def dataset_creator(folder: Path):
  bbox_folder = folder / 'box'
  entities_folder = folder / 'entities'
  img_folder = folder / 'img'

  # Sort by filename so that when zipping them together
  # we don't get some other file (just in case)
  entities_files = sorted(entities_folder.glob("*.txt"))
  bbox_files = sorted(bbox_folder.glob("*.txt"))
  img_files = sorted(img_folder.glob("*.jpg"))

  data = []

  print("Reading dataset:")
  for bbox_file, entities_file, img_file in tqdm(zip(bbox_files, entities_files, img_files), total=len(bbox_files)):            
    # Read the files
    bbox = read_bbox_and_words(bbox_file)
    entities = read_entities(entities_file)
    image = Image.open(img_file)

    # Assign labels to lines in bbox using entities
    bbox_labeled = assign_labels(bbox, entities)
    del bbox

    # Split lines into separate tokens
    new_bbox_l = []
    for index, row in bbox_labeled.iterrows():
      new_bbox_l += split_line(row)
    new_bbox = pd.DataFrame(new_bbox_l, columns=bbox_labeled.columns, dtype=np.int16)
    del bbox_labeled


    # Do another label assignment to keep the labeling more precise 
    for index, row in new_bbox.iterrows():
      label = row['label']

      if label != "O":
        entity_values = entities.iloc[0, entities.columns.get_loc(label.lower())]
        entity_set = entity_values.split()
        
        if any(SequenceMatcher(a=row['line'], b=b).ratio() > 0.7 for b in entity_set):
            label = "S-" + label
        else:
            label = "O"
      
      new_bbox.at[index, 'label'] = label

    width, height = image.size
  
    data.append([new_bbox, width, height])

  return data

Vì tập dữ liệu có hai thư mục, một thư mục dùng để huấn luyện mô hình và một thư mục để kiểm tra hiệu suất của nó, chúng ta có thể sử dụng cùng một tập lệnh để đọc cả hai và lưu chúng trong các biến được tôn trọng của chúng.

In [ ]:
dataset_train = dataset_creator(sroie_folder_path / 'train')
dataset_test = dataset_creator(sroie_folder_path / 'test')

## Viết tập dữ liệu đã chuyển đổi
Bây giờ chúng tôi đã chuyển đổi tập dữ liệu của mình thành một định dạng mà mô hình có thể hiểu để đào tạo, chúng tôi cần lưu mọi thứ vào tệp.

### Xác định chức năng viết
Chúng tôi sẽ sử dụng chức năng tương tự để ghi vào tệp tàu và tệp thử nghiệm.

Chức năng chuẩn hóa có nghĩa là chuẩn hóa các điểm hộp giới hạn trong một phạm vi [0,1000] bằng cách sử dụng chiều rộng và chiều cao của hình ảnh biên lai [\ [source \]] (https://huggingface.co/transformers/model_doc/ layoutlm.html # tổng quan).

In [ ]:
def normalize(points: list, width: int, height: int) -> list:
  x0, y0, x2, y2 = [int(p) for p in points]
  
  x0 = int(1000 * (x0 / width))
  x2 = int(1000 * (x2 / width))
  y0 = int(1000 * (y0 / height))
  y2 = int(1000 * (y2 / height))

  return [x0, y0, x2, y2]


def write_dataset(dataset: list, output_dir: Path, name: str):
  print(f"Writing {name}ing dataset:")
  with open(output_dir / f"{name}.txt", "w+", encoding="utf8") as file, \
       open(output_dir / f"{name}_box.txt", "w+", encoding="utf8") as file_bbox, \
       open(output_dir / f"{name}_image.txt", "w+", encoding="utf8") as file_image:

      # Go through each dataset
      for datas in tqdm(dataset, total=len(dataset)):
        data, width, height = datas
        
        filename = data.iloc[0, data.columns.get_loc('filename')]

        # Go through every row in dataset
        for index, row in data.iterrows():
          bbox = [int(p) for p in row[['x0', 'y0', 'x2', 'y2']]]
          normalized_bbox = normalize(bbox, width, height)

          file.write("{}\t{}\n".format(row['line'], row['label']))
          file_bbox.write("{}\t{} {} {} {}\n".format(row['line'], *normalized_bbox))
          file_image.write("{}\t{} {} {} {}\t{} {}\t{}\n".format(row['line'], *bbox, width, height, filename))

        # Write a second newline to separate dataset from others
        file.write("\n")
        file_bbox.write("\n")
        file_image.write("\n")

In [ ]:
dataset_directory = Path('/kaggle/working','dataset')

dataset_directory.mkdir(parents=True, exist_ok=True)

write_dataset(dataset_train, dataset_directory, 'train')
write_dataset(dataset_test, dataset_directory, 'test')

# Creating the 'labels.txt' file to the the model what categories to predict.
labels = ['COMPANY', 'DATE', 'ADDRESS', 'TOTAL']
IOB_tags = ['S']
with open(dataset_directory / 'labels.txt', 'w') as f:
  for tag in IOB_tags:
    for label in labels:
      f.write(f"{tag}-{label}\n")
  # Writes in the last label O - meant for all non labeled words
  f.write("O")

# 2. Tinh chỉnh LayoutLM
Chúng tôi đã tải xuống và chuyển đổi tập dữ liệu của mình thành một tập hợp có thể đào tạo và kiểm tra được, bây giờ chúng tôi có thể bắt đầu tinh chỉnh mô hình.

## Tải xuống mô hình
Đầu tiên, chúng tôi sẽ sao chép dự án LayoutLM Github chứa tập lệnh để tinh chỉnh mô hình của chúng tôi.

In [ ]:
%%bash
git clone https://github.com/microsoft/unilm.git
cd unilm/layoutlm/deprecated
pip install .

## Training

In [ ]:
pretrained_model_folder_input= sroie_folder_path / Path('layoutlm-base-uncased') # Define it so we can copy it into our working directory

pretrained_model_folder=Path('/kaggle/working/layoutlm-base-uncased/') 
label_file=Path(dataset_directory, "labels.txt")

# Move to the script directory
os.chdir("/kaggle/working/unilm/layoutlm/deprecated/examples/seq_labeling")

Đầu tiên, tôi sẽ sao chép mô hình cơ sở được đào tạo trước vào thư mục làm việc của chúng tôi để thay đổi tệp cấu hình của nó. Tôi chỉ thay đổi số lượng tiêu đề chú ý từ ** 16 ** thành ** 12 **, vì đó là kích thước ban đầu.

In [ ]:
! cp -r "{pretrained_model_folder_input}" "{pretrained_model_folder}"
! sed -i 's/"num_attention_heads": 16,/"num_attention_heads": 12,/' "{pretrained_model_folder}/"config.json

In [ ]:
! cat "/kaggle/working/layoutlm-base-uncased/config.json"

In [ ]:
! rm -rf /kaggle/working/dataset/cached*

In [ ]:
! python run_seq_labeling.py \
                            --data_dir /kaggle/working/dataset \
                            --labels /kaggle/working/dataset/labels.txt \
                            --model_name_or_path "{pretrained_model_folder}" \
                            --model_type layoutlm \
                            --max_seq_length 512 \
                            --do_lower_case \
                            --do_train \
                            --num_train_epochs 10 \
                            --logging_steps 50 \
                            --save_steps -1 \
                            --output_dir output \
                            --overwrite_output_dir \
                            --per_gpu_train_batch_size 8 \
                            --per_gpu_eval_batch_size 16

## Predicting

In [ ]:
# Evaluate for test set and make predictions
! python run_seq_labeling.py \
                            --data_dir /kaggle/working/dataset \
                            --labels /kaggle/working/dataset/labels.txt \
                            --model_name_or_path "{pretrained_model_folder}" \
                            --model_type layoutlm \
                            --do_lower_case \
                            --max_seq_length 512 \
                            --do_predict \
                            --logging_steps 10 \
                            --save_steps -1 \
                            --output_dir output \
                            --per_gpu_eval_batch_size 8

In [ ]:
cat output/test_results.txt

# Mẫu kết quả
Ví dụ cho thấy hai hình ảnh cạnh nhau của cùng một biên lai, trong đó các hộp màu là các đường được dán nhãn. Bên trái là * gốc *, vì vậy dữ liệu chúng tôi đã gắn nhãn và bên phải là dự đoán của mô hình.

In [ ]:
import cv2
from matplotlib import pyplot, patches
import matplotlib

data = pd.read_csv("/kaggle/working/dataset/test_image.txt", delimiter="\t", names=["name", "bbox", "size", "image"])
data_category = pd.read_csv("/kaggle/working/dataset/test.txt", delimiter="\t", names=["name", "true_category"]).drop(columns=["name"])
data_prediction_category = pd.read_csv("output/test_predictions.txt", delimiter=" ", names=["name", "prediction_category"]).drop(columns=["name"])

data_merge = data.merge(data_category, left_index=True, right_index=True)
merged = data_merge.merge(data_prediction_category, left_index=True, right_index=True)
merged_groups = list(merged.groupby("image"))

In [ ]:
def display_prediction(data, file):
  colors = {
      "S-TOTAL": (255,0,0),
      "S-DATE": (0,255,0),
      "S-ADDRESS": (0,0, 255),
      "S-COMPANY": (255,255,0),
      "O": (192,192,192)
  }


  imagename = data[0].split(".")[0] + ".jpg"
  print("Filename:",imagename)
  image_path = str(sroie_folder_path / 'test' / 'img' / imagename)

  img=cv2.imread(image_path)
  img_prediction=cv2.imread(image_path)

  data = data[1]
  for bbox, category, prediction_category in zip(data['bbox'], data['true_category'], data['prediction_category']):
    (x1, y1, x2, y2) = [int(coordinate) for coordinate in bbox.split()]

    img_prediction = cv2.rectangle(img_prediction, (x1, y1), (x2, y2), colors[prediction_category], 2 if "O" in prediction_category else 4)
    img = cv2.rectangle(img, (x1, y1), (x2, y2), colors[category], 2 if "O" in category else 4)

  matplotlib.rcParams['figure.figsize'] = 15 ,18

  cv2.imwrite("prediction.jpg", img_prediction)

  # Plot
  fig, ax = matplotlib.pyplot.subplots(1,2)
  ax[0].set_title("Original", fontsize= 30)
  ax[0].imshow(img);
  ax[1].set_title("Prediction", fontsize= 30)
  ax[1].imshow(img_prediction);

  # Legend
  handles = [
      patches.Patch(color='yellow', label='Company'),
      patches.Patch(color='blue', label='Address'),
      patches.Patch(color='green', label='Date'),
      patches.Patch(color='red', label='Total'),
      patches.Patch(color='gray', label='Other')
  ]

  fig.legend(handles=handles, prop={'size': 25}, loc='lower center')

Đây là một ví dụ khi quá trình xử lý trước của tôi không hoàn hảo, nhưng mô hình vẫn dự đoán kết quả chính xác. Từ đó, chúng ta có thể thấy rằng nếu quá trình xử lý trước của tôi tốt hơn, thì mô hình sẽ có điểm chính xác tốt hơn.

In [ ]:
display_prediction(merged_groups[0], 'test')

In [ ]:
display_prediction(merged_groups[34], 'test')